## Sparrow Demo

Envirement requirements:
- docker
- git
- github
- java
- jupyter: spark\tensorflow

In [7]:
import sys
sys.path.append("..")

import utils
import time

# bootstrap
bootstrap = utils.Bootstrap("../worksapce.config.json")
bootstrap.run()

biz_tags = [{"Key": "id", "Value": "rec-demo"}]


def start_notebook(instance_type="ecs.g6.large", notebook="tensorflow-notebook"):
    aliyun_ecs_manager = utils.AliyunECSManager()
    cmd_generator = utils.CommandGenerator()

    # 创建ECS实例的设置
    settings = aliyun_ecs_manager.create_default_settings()
    settings.set_instance_type(instance_type)
    
    biz_tags = [{'Key': 'Project', 'Value': 'JupyterNotebook'}]
    settings.set_biz_tags(biz_tags)
    
    # 创建ECS实例
    vm_property = aliyun_ecs_manager.create_instance(settings)

    # 等待ECS实例启动和初始化
    time.sleep(20)  # 可能需要更长时间确保ECS实例完全启动

    # 生成并执行设置环境的命令
    cmd_content = "\n".join([
        cmd_generator.generate_install_docker_command(),
        cmd_generator.generate_install_git_command(),
        cmd_generator.generate_config_github_command(),
        cmd_generator.generate_start_jupyter_command(name=notebook)
    ])

    aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id, cmd_content=cmd_content)

    return aliyun_ecs_manager.get_instances(biz_tags)[0]

### 1. Spark notebook

In [ ]:
vm = start_notebook(instance_type="ecs.g6.large", notebook="spark-notebook")

### 2. Tensorflow notebook

In [8]:
vm = start_notebook(instance_type="ecs.g6.large", notebook="tensorflow-notebook")

In [10]:
login_cmd = """
ssh -o UserKnownHostsFile=/dev/null root@{server}
""".format(server=vm.public_ip)

print(login_cmd)


ssh -o UserKnownHostsFile=/dev/null root@None



### 2. login the instance

### 3. Clone code
```bash
mkdir code
cd code
git clone git@github.com:dream-365/SparrowRecSys.git
```

In [ ]:
clone_code_cmd = """
mkdir work
cd work
git clone git@github.com:dream-365/SparrowRecSys.git
"""

aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id,
                                  cmd_content=clone_code_cmd)

### 4. Compile prject
```bash
cd SparrowRecSys
docker run -it --rm --name sparrow-maven-project \
-v "$(pwd)":/usr/src/sparrow \
-w /usr/src/sparrow maven:3.3-jdk-8 \
mvn package
```

In [ ]:
compile_code_cmd = """
cd work/SparrowRecSys
docker run -it --rm --name sparrow-maven-project \
-v "$(pwd)":/usr/src/sparrow \
-w /usr/src/sparrow maven:3.3-jdk-8 \
mvn package
"""

aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id,
                                  cmd_content=compile_code_cmd)

### 5. Run project

```bash
docker run --rm \
-v "$PWD":/usr/src/sparrow \
-p 9010:6010 \
-w /usr/src/sparrow openjdk:11 \
java -cp target/SparrowRecSys-1.0-SNAPSHOT-jar-with-dependencies.jar com.sparrowrecsys.online.RecSysServer
```

In [ ]:
run_project_cmd = """
cd work/SparrowRecSys
docker run -d --rm \
-v "$PWD":/usr/src/sparrow \
-p 9010:6010 \
-w /usr/src/sparrow openjdk:11 \
java -cp target/SparrowRecSys-1.0-SNAPSHOT-jar-with-dependencies.jar com.sparrowrecsys.online.RecSysServer
"""

aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id,
                                  cmd_content=run_project_cmd)

### 7. Get recommendation

e.g:
```
http://47.236.229.233:9010/getrecforyou?id=427&size=32&model=emb
```

### 6. Clenup 

In [4]:
aliyun_ecs_manager = utils.AliyunECSManager()
aliyun_ecs_manager.clean(biz_tags)